In [1]:
import hashlib
import inspect
import types

In [2]:
def fdigest(filename):
    f = open(filename,mode='rb')
    m = hashlib.sha256(f.read())
    f.close()
    return m.hexdigest()

In [3]:
h = fdigest('Loads.ipynb')
h

'3b93953100507915b0198cd96d35331a56d82edf3af433c1da3a1fe6e9f6e070'

In [4]:
def extend(old):
    """This is used as a class decorator to 'extend' class definitions,
    for example, over widely dispersed areas.  EG:

        class Foo(object):
            . . .
        @extend(Foo)
        class Foo:
            def meth2(...):
            . . .

    will result in one class Foo containing all methods, etc."""

    def _extend(new,old=old):
        if new.__name__ != old.__name__:
            raise TypeError("Class names must match: '{}' != '{}'".format(new.__name__,old.__name__))
        ok = ['__init__']
        for a,v in inspect.getmembers(new):
            if not a.startswith('_') or a in ok:
                if type(v) is types.MethodType:
                    v = types.MethodType(v.im_func,old if v.im_self is new else v.im_self,old)
                elif type(v) is property:
                    v = property(v.fget,v.fset,v.fdel)
                elif type(v) is types.FunctionType:
                    v = staticmethod(types.FunctionType(v.func_code,v.func_globals,v.func_name,v.func_defaults,v.func_closure))
                setattr(old,a,v)
                #print('Set {} in class {} to type {}'.format(a,old.__name__,type(v)))
        return old
    
    return _extend

In [6]:
class Foo(object):
    def im(s):
        return s
    @classmethod
    def cm(s):
        return s

In [8]:
l = inspect.getmembers(Foo)
l

[('__class__', type),
 ('__delattr__', <slot wrapper '__delattr__' of 'object' objects>),
 ('__dict__',
  <dictproxy {'__dict__': <attribute '__dict__' of 'Foo' objects>,
   '__doc__': None,
   '__module__': '__main__',
   '__weakref__': <attribute '__weakref__' of 'Foo' objects>,
   'cm': <classmethod at 0xb4ba0c8c>,
   'im': <function __main__.im>}>),
 ('__doc__', None),
 ('__format__', <method '__format__' of 'object' objects>),
 ('__getattribute__', <slot wrapper '__getattribute__' of 'object' objects>),
 ('__hash__', <slot wrapper '__hash__' of 'object' objects>),
 ('__init__', <slot wrapper '__init__' of 'object' objects>),
 ('__module__', '__main__'),
 ('__new__', <function __new__>),
 ('__reduce__', <method '__reduce__' of 'object' objects>),
 ('__reduce_ex__', <method '__reduce_ex__' of 'object' objects>),
 ('__repr__', <slot wrapper '__repr__' of 'object' objects>),
 ('__setattr__', <slot wrapper '__setattr__' of 'object' objects>),
 ('__sizeof__', <method '__sizeof__' of 'ob

In [9]:
Foo.cm2 = types.MethodType?

In [10]:
def _cm2(c):
    return 'cm2',c
Foo.cm2 = types.MethodType(_cm2,Foo,Foo)

In [11]:
l = inspect.getmembers(Foo)
l

[('__class__', type),
 ('__delattr__', <slot wrapper '__delattr__' of 'object' objects>),
 ('__dict__',
  <dictproxy {'__dict__': <attribute '__dict__' of 'Foo' objects>,
   '__doc__': None,
   '__module__': '__main__',
   '__weakref__': <attribute '__weakref__' of 'Foo' objects>,
   'cm': <classmethod at 0xb4ba0c8c>,
   'cm2': <bound method Foo._cm2 of <class '__main__.Foo'>>,
   'im': <function __main__.im>}>),
 ('__doc__', None),
 ('__format__', <method '__format__' of 'object' objects>),
 ('__getattribute__', <slot wrapper '__getattribute__' of 'object' objects>),
 ('__hash__', <slot wrapper '__hash__' of 'object' objects>),
 ('__init__', <slot wrapper '__init__' of 'object' objects>),
 ('__module__', '__main__'),
 ('__new__', <function __new__>),
 ('__reduce__', <method '__reduce__' of 'object' objects>),
 ('__reduce_ex__', <method '__reduce_ex__' of 'object' objects>),
 ('__repr__', <slot wrapper '__repr__' of 'object' objects>),
 ('__setattr__', <slot wrapper '__setattr__' of 'o

In [15]:
Foo.cm(), Foo.cm2()

(__main__.Foo, ('cm2', __main__.Foo))

In [16]:
class Bar(Foo):
    pass

In [17]:
Bar.cm(), Bar.cm2()

(__main__.Bar, ('cm2', __main__.Foo))